# Using dataobject-based classes for creating common datatypes

In [1]:
from recordclass import dataobject, clsconfig
from recordclass._linkedlist import linkedlist
import sys

## LinkedList

In [2]:
class LinkedItem(dataobject, fast_new=True):
    val: object
    next: 'LinkedItem'

# @clsconfig(deep_dealloc=True)
class LinkedList(dataobject, deep_dealloc=True):
    start: LinkedItem = None
    end: LinkedItem = None
        
    def append(self, val):
        link = LinkedItem(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link
        
#     def __del__(self):
#         curr = self.start
#         while curr is not None:
#             next = curr.next
#             curr.next = None
#             curr = next

    def __iter__(self):
        return IterLinkedList(self)

class IterLinkedList(dataobject, fast_new=True):
    node: LinkedItem
        
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [3]:
def make_llist(N):
    ll = LinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [4]:
class LinkedItem2:
    __slots__ = 'val', 'next'

    def __init__(self, val, next):
        self.val = val
        self.next = next
    
class LinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = LinkedItem2(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self)

class IterLinkedList2:
    __slots__ = 'node',
    
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val

In [5]:
def make_llist2(N):
    ll = LinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [6]:
N = 1000000
Mb = 1000000

In [7]:
%time ll1 = make_llist(N)
%timeit make_llist(N)

CPU times: user 253 ms, sys: 3.95 ms, total: 257 ms
Wall time: 256 ms
219 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%time s1 = sum(ll1)
%timeit sum(ll1)
print(s1)

CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 139 ms
123 ms ± 6.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
499999500000


In [9]:
print(sys.getsizeof(ll1.start), sys.getsizeof(ll1))
M1 = sys.getsizeof(ll1) + N * sys.getsizeof(ll1.start)
print('Memory footprint: %.2f Мб' % (M1/1000000))
# del ll1

32 32
Memory footprint: 32.00 Мб


In [10]:
%time ll2 = make_llist2(N)
%timeit make_llist2(N)

CPU times: user 703 ms, sys: 31.7 ms, total: 735 ms
Wall time: 733 ms
343 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%time s2 = sum(ll2)
%timeit sum(ll2)
print(s2)

CPU times: user 148 ms, sys: 0 ns, total: 148 ms
Wall time: 146 ms
136 ms ± 282 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
499999500000


In [12]:
print(sys.getsizeof(ll2.start), sys.getsizeof(ll2))
M2 = sys.getsizeof(ll2) + N * sys.getsizeof(ll2.start)
print('Memory footprint: %.2f Мб' % (M2/1000000))
# del ll2

48 48
Memory footprint: 48.00 Мб


In [13]:
print(100*M1/M2)

66.66666666666667


In [14]:
def make_llist3(N):
    ll = linkedlist()
    for i in range(N):
        ll.append(i)
    return ll

In [15]:
%time ll3 = make_llist3(N)
%timeit make_llist3(N)

CPU times: user 104 ms, sys: 16 ms, total: 120 ms
Wall time: 118 ms
95.3 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%time s3 = sum(ll3)
%timeit sum(ll3)
print(s3)

CPU times: user 21.3 ms, sys: 55 µs, total: 21.4 ms
Wall time: 21.1 ms
10.3 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
499999500000


In [17]:
M3 = sys.getsizeof(ll3) + N * sys.getsizeof(ll3.start)
print('Memory footprint: %.2f Мб' % (M3/1000000))
# del ll3

Memory footprint: 32.00 Мб


## Double Linked List

In [18]:
class DLinkedItem(LinkedItem, fast_new=True):
#     val: object
#     next: 'DLinkedItem'
    prev: 'DLinkedItem'

# @clsconfig(deep_dealloc=True)
class DLinkedList(LinkedList, deep_dealloc=True):
#     start: DLinkedItem = None
#     end: DLinkedItem = None
        
    def append(self, val):
        link = DLinkedItem(val, None, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        link.prev = self.end
        self.end = link

    def __iter__(self):
        return IterLinkedList(self)

class IterLinkedList(dataobject, fast_new=True):
    node: DLinkedItem
        
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [19]:
def make_dllist(N):
    ll = DLinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [20]:
class DLinkedItem2(LinkedItem2):
    __slots__ = 'prev',

    def __init__(self, val, next, prev):
        self.val = val
        self.next = next
        self.prev = prev
    
class DLinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = DLinkedItem2(val, None, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        link.prev = self.end
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self)

# class DIterLinkedList2:
#     __slots__ = 'node',
    
#     def __init__(self, ll):
#         self.node = ll.start
    
#     def __next__(self):
#         node =  self.node
#         if node is None:
#             raise StopIteration
        
#         val = node.val
#         self.node = node.next
#         return val

In [21]:
def make_dllist2(N):
    ll = DLinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [22]:
%time dll1 = make_dllist(N)
%timeit make_dllist(N)

CPU times: user 268 ms, sys: 8.14 ms, total: 276 ms
Wall time: 276 ms
261 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%time ds1 = sum(dll1)
%timeit sum(dll1)
print(ds1)

CPU times: user 121 ms, sys: 3.77 ms, total: 125 ms
Wall time: 123 ms
125 ms ± 7.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
499999500000


In [24]:
print(sys.getsizeof(dll1.start), sys.getsizeof(dll1))
M1 = sys.getsizeof(dll1) + N * sys.getsizeof(dll1.start)
print('Memory footprint: %.2f Мб' % (M1/1000000))
# del ll1

40 32
Memory footprint: 40.00 Мб


In [25]:
%time dll2 = make_dllist2(N)
%timeit make_dllist2(N)

CPU times: user 818 ms, sys: 11.9 ms, total: 830 ms
Wall time: 829 ms
448 ms ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%time ds2 = sum(dll2)
%timeit sum(dll2)
print(ds2)

CPU times: user 152 ms, sys: 121 µs, total: 152 ms
Wall time: 150 ms
137 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
499999500000


In [27]:
print(sys.getsizeof(dll2.start), sys.getsizeof(dll2))
M2 = sys.getsizeof(dll2) + N * sys.getsizeof(dll2.start)
print('Memory footprint: %.2f Мб' % (M2/1000000))
# del ll2

56 48
Memory footprint: 56.00 Мб


In [28]:
print(100*M1/M2)

71.42856734694227
